In [1]:
# =============================================================================
# XGBoost CPU 파이프라인 (pandas + sklearn)
# =============================================================================
# 용도: 50pct 이상 대용량 데이터 또는 GPU 미사용 환경
# 셀 구분: #%% 주석으로 노트북 셀 분리
# =============================================================================
#
# ⚠️ TODO: 과적합 해결 (Train ROC-AUC 0.89 vs Valid 0.73)
# ─────────────────────────────────────────────────────────
#
# [1] CFG.XGB_PARAMS에 정규화 강화 (Ridge/Lasso)
#     CFG.XGB_PARAMS['reg_lambda'] = 5.0    # Ridge (L2), 기본=1 → 5~10 시도
#     CFG.XGB_PARAMS['reg_alpha'] = 1.0     # Lasso (L1), 기본=0 → 0.5~2 시도
#
# [2] CFG.XGB_PARAMS에 scale_pos_weight 추가 (클래스 불균형 보정)
#     # 다운샘플링 후 1:1이면 불필요하지만, 원본 비율 반영 시:
#     CFG.XGB_PARAMS['scale_pos_weight'] = len(train_df[train_df['clicked']==0]) / len(train_df[train_df['clicked']==1])
#
# [3] CFG.XGB_PARAMS 트리 복잡도 줄이기
#     CFG.XGB_PARAMS['max_depth'] = 6       # 현재 8 → 5~6으로 줄이기
#     CFG.XGB_PARAMS['min_child_weight'] = 5 # 기본=1 → 5~10 (leaf 최소 샘플)
#     CFG.XGB_PARAMS['gamma'] = 0.3         # 기본=0 → 0.1~0.5 (split 최소 gain)
#
# [4] 피처 수 줄이기 (현재 1437개 → OHE 때문에 큼)
#     # interaction 피처 제거 테스트:
#     # add_interaction_features() 호출 주석 처리 후 재실행
#     # 또는 groupby 피처만으로 테스트
#
# ─────────────────────────────────────────────────────────

In [2]:
# import json
# # 저장된 파일 불러오기
# with open('XGB_CPU_20240216_0105_config.json', 'r', encoding='utf-8') as f:
#     config = json.load(f)
# print(config['learning_rate'])  # 0.08 출력
# print(config['max_depth'])      # 6 출력

[PART 1] 라이브러리 임포트

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
import os
import sys
import random
import gc
import datetime
from itertools import combinations
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    precision_recall_fscore_support,
    precision_recall_curve,
    auc,
    log_loss,
    accuracy_score
)
import warnings

warnings.filterwarnings('ignore')

# seq 임베딩 경로 추가
sys.path.append('/home/konkukstat/python3/HB/KSH')



[PART 2] 설정 및 유틸리티 함수

In [ ]:
class CFG:
    DATA_PATH = '/home/konkukstat/python3/workspace/data'
    TRAIN_FILE = 'train.parquet'   # 변경 가능
    TEST_FILE = 'test.parquet'

    # [수정] 현재 시간을 구해서 파일명 접두사에 붙입니다.
    # 예: XGB_CPU_20240214_2040
    NOW = datetime.datetime.now().strftime('%Y%m%d_%H%M') 
    OUTPUT_PREFIX = f'XGB_CPU_{NOW}'
    OUTPUT_DIR = os.path.join('results', OUTPUT_PREFIX)

    SEEDS = [42, 106, 1031]
    NEG_RATIO = 1
    VALID_RATIO = 0.2

    XGB_PARAMS = {
        'objective': 'binary:logistic',
        'eval_metric': 'aucpr',
        'learning_rate': 0.08,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'n_estimators': 500,
        'early_stopping_rounds': 30,
        'tree_method': 'hist',
        'min_child_weight' : 3,
        'reg_lambda' : 5,
        'reg_alpha' : 1,
        'device': 'cuda',
    }

    # Feature Engineering Flags (True: 사용 / False: 미사용)
    USE_SEQ_FEATURES = False
    USE_SEQ_FEATURES_V2 = True    # seq 심층 파생변수 (nunique, click_score, repeat_ratio)
    USE_INTERACTION_FEATURES = True
    USE_GROUPBY_FEATURES = True
    USE_COUNT_FEATURES = True

    # 범주형 변수 목록 (unique.db 기반)
    CATEGORICAL_COLS = [
        'gender', 'age_group', 'inventory_id', 'day_of_week', 'hour',
        'l_feat_1', 'l_feat_2', 'l_feat_3', 'l_feat_4', 'l_feat_8',
        'l_feat_13', 'l_feat_16', 'l_feat_18', 'l_feat_19', 'l_feat_20',
        'l_feat_21', 'l_feat_22', 'l_feat_23', 'l_feat_24', 'l_feat_26',
        'l_feat_27', 'feat_e_4', 'feat_d_1', 'feat_d_2', 'feat_d_3',
        'feat_d_5', 'feat_d_6', 'feat_a_1', 'feat_a_2', 'feat_a_3',
        'feat_a_4', 'feat_a_6', 'feat_a_7', 'feat_a_8', 'feat_a_9',
        'feat_a_10', 'feat_a_11', 'feat_a_12', 'feat_a_13', 'feat_a_15',
        'feat_a_16', 'feat_a_17', 'feat_a_18'
    ]

     # 전체 수치형 컬럼 목록 (119개 - 범주형 - seq - clicked)
    ALL_NUMERIC_COLS = [
        'l_feat_5', 'l_feat_6', 'l_feat_7', 'l_feat_9', 'l_feat_10',
        'l_feat_11', 'l_feat_12', 'l_feat_14', 'l_feat_15', 'l_feat_17',
        'l_feat_25',
        'feat_e_1', 'feat_e_2', 'feat_e_3', 'feat_e_5', 'feat_e_6',
        'feat_e_7', 'feat_e_8', 'feat_e_9', 'feat_e_10',
        'feat_d_4',
        'feat_c_1', 'feat_c_2', 'feat_c_3', 'feat_c_4', 'feat_c_5',
        'feat_c_6', 'feat_c_7', 'feat_c_8',
        'feat_b_1', 'feat_b_2', 'feat_b_3', 'feat_b_4', 'feat_b_5',
        'feat_b_6',
        'feat_a_5', 'feat_a_14',
        'history_a_1', 'history_a_2', 'history_a_3', 'history_a_4',
        'history_a_5', 'history_a_6', 'history_a_7',
        'history_b_1', 'history_b_2', 'history_b_3', 'history_b_4',
        'history_b_5', 'history_b_6', 'history_b_7', 'history_b_8',
        'history_b_9', 'history_b_10', 'history_b_11', 'history_b_12',
        'history_b_13', 'history_b_14', 'history_b_15', 'history_b_16',
        'history_b_17', 'history_b_18', 'history_b_19', 'history_b_20',
        'history_b_21', 'history_b_22', 'history_b_23', 'history_b_24',
        'history_b_25', 'history_b_26', 'history_b_27', 'history_b_28',
        'history_b_29', 'history_b_30',
    ]
    
    DEFAULT_AGG_DICT = {col: ['mean', 'std'] for col in ALL_NUMERIC_COLS}

    # EXCLUDE = {*CATEGORICAL_COLS, 'seq', 'clicked'}

    # # Groupby 통계에 사용할 agg_dict (기본값)

    # DEFAULT_AGG_DICT = {
    # col: ['mean', 'std'] 
    # for col in train_df.columns 
    # if col not in EXCLUDE
    #}

    # DEFAULT_AGG_DICT = {
    #     'history_a_1': ['mean', 'std'], 'history_a_2': ['mean', 'std'],
    #     'history_a_3': ['mean', 'std'], 'history_a_6': ['mean', 'std'],
    #     'feat_e_2': ['mean', 'std'], 'feat_e_3': ['mean', 'std'],
    #     'feat_c_8': ['mean', 'std'], 'feat_e_9': ['mean', 'std'],
    #     'feat_d_4': ['mean', 'std'],
    #     'l_feat_1': ['mean', 'std'], 'l_feat_2': ['mean', 'std'],
    #     'l_feat_5': ['mean', 'std'], 'l_feat_7': ['mean', 'std'],
    #     'l_feat_10': ['mean', 'std'], 'l_feat_15': ['mean', 'std'],
    # }

    

def seed_everything(seed: int):
    """모든 난수 생성기를 고정하여 재현 가능한 결과 보장"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)


def save_experiment_config(cfg):
    """설정값(파라미터, 플래그 등)을 JSON 파일로 저장"""
    import json
    config_data = {}
    
    # XGB_PARAMS 저장
    if hasattr(cfg, 'XGB_PARAMS'):
        config_data.update(cfg.XGB_PARAMS)
        
    # Feature Flags 및 기타 설정 저장
    for attr in dir(cfg):
        if attr.startswith('USE_') or attr in ['VALID_RATIO', 'NEG_RATIO', 'SEEDS', 'OUTPUT_PREFIX']:
            val = getattr(cfg, attr)
            # serialize numpy types
            if hasattr(val, 'item'): val = val.item()
            config_data[attr] = val
            
    def default(o):
        if hasattr(o, 'item'): return o.item()
        return str(o)
    
    filename = os.path.join(cfg.OUTPUT_DIR, f"{cfg.OUTPUT_PREFIX}_config.json")
    
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(config_data, f, indent=4, ensure_ascii=False, default=default)
        print(f"📄 Configuration saved: {filename}")
    except Exception as e:
        print(f"⚠️ Failed to save configuration: {e}")


[PART 3] 피처 엔지니어링 함수

In [5]:
# --- 클릭 확률 맵 로드 ---
try:
    _df_click_prob = pd.read_excel(os.path.join(CFG.DATA_PATH, 'high_click_numbers.xlsx'))
    CLICK_PROB_MAP = dict(zip(_df_click_prob['number'], _df_click_prob['click_prob']))
except FileNotFoundError:
    print("⚠️ 'high_click_numbers.xlsx' not found. click_prob features will be 0.")
    CLICK_PROB_MAP = {}

POS_LIST = frozenset({370, 528, 68, 561, 144, 227, 417, 442, 186, 395})
NEG_LIST = frozenset({154, 222, 84, 498, 434, 511, 216, 497, 309, 446})



def add_seq_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    seq 컬럼에서 파생 피처 4개 생성:
      seq_len, avg_click_prob, seq_neglogcount, seq_poslogcount
    """
    seq_col = df["seq"]

    seq_len = np.zeros(len(df), dtype=np.int32)
    avg_prob = np.zeros(len(df), dtype=np.float32)
    seq_neg = np.zeros(len(df), dtype=np.int32)
    seq_pos = np.zeros(len(df), dtype=np.int32)

    for i, s in enumerate(seq_col):
        if isinstance(s, str) and s:
            arr = [int(x) for x in s.split(",") if x]
            seq_len[i] = len(arr)
            if CLICK_PROB_MAP:
                probs = [CLICK_PROB_MAP.get(num, 0.0) for num in arr]
                avg_prob[i] = np.mean(probs) if probs else 0.0
            seq_neg[i] = sum(1 for x in arr if x in NEG_LIST)
            seq_pos[i] = sum(1 for x in arr if x in POS_LIST)

    df["seq_len"] = seq_len
    df["avg_click_prob"] = avg_prob

    df["seq_neglogcount"] = seq_neg
    df["seq_poslogcount"] = seq_pos
    
    return df


def build_item_ctr_from_train(train_df: pd.DataFrame) -> dict:
    """
    train의 seq + clicked 컬럼으로 아이템별 CTR 딕셔너리를 생성.
    CTR = clicked=1에서의 등장 빈도 / 전체 등장 빈도
    """
    from collections import Counter
    pos_counter = Counter()
    neg_counter = Counter()

    for clicked_val, counter in [(1, pos_counter), (0, neg_counter)]:
        subset = train_df[train_df['clicked'] == clicked_val]['seq']
        for s in subset:
            if isinstance(s, str) and s:
                items = s.split(',')
                counter.update(items)

    all_items = set(pos_counter.keys()) | set(neg_counter.keys())
    item_ctr = {}
    for item in all_items:
        freq_1 = pos_counter.get(item, 0)
        freq_0 = neg_counter.get(item, 0)
        total = freq_1 + freq_0
        item_ctr[item] = freq_1 / total if total > 0 else 0.0

    print(f"  ✅ Item CTR map built: {len(item_ctr):,} items")
    return item_ctr


def add_seq_features_v2(df: pd.DataFrame, item_ctr_map: dict) -> pd.DataFrame:
    """
    seq 컬럼에서 심층 파생 피처 3개 생성 (Cohen's d 상위 3개):
      seq_nunique      : 고유 아이템 수 (탐색 다양성)
      seq_click_score  : 아이템별 CTR 평균 (클릭 경향 점수)
      seq_repeat_ratio : 반복 비율 (관심 집중도)
    """
    seq_col = df["seq"]

    nunique = np.zeros(len(df), dtype=np.int32)
    click_score = np.zeros(len(df), dtype=np.float32)
    repeat_ratio = np.zeros(len(df), dtype=np.float32)

    for i, s in enumerate(seq_col):
        if isinstance(s, str) and s:
            items = s.split(",")
            seq_len = len(items)
            unique_count = len(set(items))

            nunique[i] = unique_count
            repeat_ratio[i] = (seq_len - unique_count) / seq_len if seq_len > 0 else 0.0

            if item_ctr_map:
                ctrs = [item_ctr_map.get(item, 0.0) for item in items]
                click_score[i] = np.mean(ctrs) if ctrs else 0.0

    df["seq_nunique"] = nunique
    df["seq_click_score"] = click_score
    df["seq_repeat_ratio"] = repeat_ratio
    return df


def add_interaction_features(df: pd.DataFrame, cols: list = None) -> pd.DataFrame:
    """범주형 피처 쌍의 조합으로 interaction 피처 생성"""
    if cols is None:
        cols = ['gender', 'age_group', 'inventory_id', 'day_of_week', 'hour']
    cols = [c for c in cols if c in df.columns]
    for col1, col2 in combinations(cols, 2):
        new_col = f'{col1}_{col2}'
        df[new_col] = df[col1].astype(str) + '_' + df[col2].astype(str)
    return df


def add_groupby_features(train_df, val_df, test_df, feature_name, agg_dict=None):
    """
    train_df 기준으로 groupby 통계 계산 → val, test에 merge.
    agg_dict 미지정 시 CFG.DEFAULT_AGG_DICT 사용.
    존재하지 않는 컬럼 및 비수치형 컬럼은 자동 스킵.
    """
    if agg_dict is None:
        agg_dict = CFG.DEFAULT_AGG_DICT

    if feature_name not in train_df.columns:
        return train_df, val_df, test_df

    # 존재하는 컬럼만 필터링 + 수치형 컬럼만 필터링
    filtered_agg = {
        k: v for k, v in agg_dict.items()
        if k in train_df.columns and pd.api.types.is_numeric_dtype(train_df[k])
    }
    if not filtered_agg:
        return train_df, val_df, test_df

    agg_stats = train_df.groupby(feature_name).agg(filtered_agg)
    new_cols = [f"{feature_name}_{col[0]}_{col[1]}" for col in agg_stats.columns]
    agg_stats.columns = new_cols
    agg_stats = agg_stats.reset_index()

    train_df = train_df.merge(agg_stats, on=feature_name, how='left')
    val_df = val_df.merge(agg_stats, on=feature_name, how='left')
    test_df = test_df.merge(agg_stats, on=feature_name, how='left')

    for col in new_cols:
        fill_val = train_df[col].mean()
        val_df[col] = val_df[col].fillna(fill_val)
        test_df[col] = test_df[col].fillna(fill_val)

    return train_df, val_df, test_df


def add_count_features(train_df, val_df, test_df, count_cols):
    """Count Encoding: train에서만 빈도 계산 → val, test에 적용."""
    for col in count_cols:
        if col not in train_df.columns:
            continue
        count_map = train_df[col].value_counts().to_dict()
        train_df[f"{col}_count"] = train_df[col].map(count_map).fillna(0).astype(int)
        val_df[f"{col}_count"] = val_df[col].map(count_map).fillna(0).astype(int)
        test_df[f"{col}_count"] = test_df[col].map(count_map).fillna(0).astype(int)
    return train_df, val_df, test_df


def pandas_hstack(matrices):
    """
    CPU용 자동 hstack 함수 (Pandas/Scipy 스타일).
    Dense Array, Pandas DataFrame, CSR Matrix 무엇이든 섞여 있어도 합쳐줌.
    """
    converted_list = []
    for mat in matrices:
        if hasattr(mat, 'tocsr'):           # scipy sparse
            converted_list.append(mat.tocsr())
        elif hasattr(mat, 'values'):        # pandas DataFrame/Series
            arr = mat.values
            if arr.ndim == 1:
                arr = arr.reshape(-1, 1)
            converted_list.append(csr_matrix(arr))
        elif isinstance(mat, np.ndarray):   # numpy array
            if mat.ndim == 1:
                mat = mat.reshape(-1, 1)
            converted_list.append(csr_matrix(mat))
        else:
            converted_list.append(csr_matrix(mat))
    return hstack(converted_list).tocsr()


[PART 4] 평가 함수

In [6]:
def compute_toss_score(y_true, y_pred):
    """
    Toss 대회 스코어 계산:
    Score = 0.5 * AP + 0.5 * (1 / (1 + WeightedLogLoss))
    """
    y_pred_clipped = np.clip(y_pred, 1e-15, 1 - 1e-15)

    N_1 = np.sum(y_true == 1)
    N_0 = np.sum(y_true == 0)
    if N_1 == 0 or N_0 == 0:
        return 0.5, 0.0, 0.0

    weights = np.where(y_true == 1, 0.5 / N_1, 0.5 / N_0)
    wll = log_loss(y_true, y_pred_clipped, sample_weight=weights)
    ap = average_precision_score(y_true, y_pred)
    score = 0.5 * ap + 0.5 * (1 / (1 + wll))
    return score, ap, wll


def evaluate_model(model, X_train, y_train, X_val, y_val, threshold=0.5):
    """
    XGBoost 모델의 평가 지표를 계산하여 DataFrame으로 반환.
    ROC-AUC, PR-AUC, Avg Precision, Precision, Recall, F1-Score, Accuracy, Log Loss, Toss Score
    """
    results = {}

    for name, X, y_true in [('Train', X_train, y_train), ('Valid', X_val, y_val)]:
        y_prob = model.predict_proba(X)[:, 1]
        y_pred_class = (y_prob >= threshold).astype(int)

        y_true_np = np.asarray(y_true)

        roc = roc_auc_score(y_true_np, y_prob)
        prec_curve, rec_curve, _ = precision_recall_curve(y_true_np, y_prob)
        pr_auc = auc(rec_curve, prec_curve)
        avg_prec = average_precision_score(y_true_np, y_prob)
        ll = log_loss(y_true_np, y_prob)
        prec, rec, f1, _ = precision_recall_fscore_support(
            y_true_np, y_pred_class, average='binary', zero_division=0
        )
        acc = accuracy_score(y_true_np, y_pred_class)
        toss, _, _ = compute_toss_score(y_true_np, y_prob)

        results[name] = {
            'ROC-AUC': roc, 'PR-AUC': pr_auc, 'Avg Precision': avg_prec,
            'Precision': prec, 'Recall': rec, 'F1-Score': f1, 'Accuracy': acc,
            'Log Loss': ll, 'Toss Score': toss
        }

    df_results = pd.DataFrame(results).T
    return df_results


def soft_voting_ensemble(prediction_files, output_file):
    """여러 시드의 예측 파일을 평균하여 앙상블"""
    if not prediction_files:
        print("❌ No prediction files found.")
        return

    merged = pd.read_csv(prediction_files[0]).rename(columns={'clicked': 'click_0'})
    for i, f in enumerate(prediction_files[1:], 1):
        d = pd.read_csv(f).rename(columns={'clicked': f'click_{i}'})
        merged = merged.merge(d, on='ID')

    click_cols = [c for c in merged.columns if c.startswith('click_')]
    merged['clicked'] = merged[click_cols].mean(axis=1)

    merged[['ID', 'clicked']].to_csv(output_file, index=False)
    print(f"🎉 Ensemble saved: {output_file}")
    print(merged.head())
    return merged


[PART 5] 메인 학습 루프

In [7]:
# ── 0. 설정 저장 ──
os.makedirs(CFG.OUTPUT_DIR, exist_ok=True)
save_experiment_config(CFG)

prediction_files = []

for seed in CFG.SEEDS:
    print(f"\n{'='*60}")
    print(f"🚀 Processing Seed: {seed}")
    print(f"{'='*60}")
    seed_everything(seed)

    # ── 1. 데이터 로드 ──
    train_path = os.path.join(CFG.DATA_PATH, CFG.TRAIN_FILE)
    print(f"📂 Loading: {train_path}")
    full_train_df = pd.read_parquet(train_path)
    print(f"✅ Train loaded: {full_train_df.shape}")

    try:
        test_df = pd.read_parquet(os.path.join(CFG.DATA_PATH, CFG.TEST_FILE))
        print(f"✅ Test loaded: {test_df.shape}")
    except FileNotFoundError:
        print("⚠️ Test not found. Creating dummy.")
        test_df = full_train_df.head(100).drop(columns=['clicked'])
        test_df['ID'] = [f'TEST_{i}' for i in range(100)]

    # ── 1-1. 범주형 변수 dtype 변환 ──
    print("🔄 Converting categorical columns to str...")
    for col in CFG.CATEGORICAL_COLS:
        if col in full_train_df.columns:
            full_train_df[col] = full_train_df[col].astype(str)
        if col in test_df.columns:
            test_df[col] = test_df[col].astype(str)
    print("✅ Categorical dtype conversion done")

    # ── 2. seq 파생 피처 생성 ──
    if CFG.USE_SEQ_FEATURES:
        print("🛠️ Adding seq features...")
        full_train_df = add_seq_features(full_train_df)
        test_df = add_seq_features(test_df)
    else:
        print("⏭️ Skipping seq features...")

    # ── 2-1. seq 심층 파생 피처 생성 (v2) ──
    if CFG.USE_SEQ_FEATURES_V2:
        print("🛠️ Building item CTR map from train seq...")
        item_ctr_map = build_item_ctr_from_train(full_train_df)
        print("🛠️ Adding seq features v2 (nunique, click_score, repeat_ratio)...")
        full_train_df = add_seq_features_v2(full_train_df, item_ctr_map)
        test_df = add_seq_features_v2(test_df, item_ctr_map)
    else:
        print("⏭️ Skipping seq features v2...")

    # ── 3. seq 컬럼 보관 후 제거 (임베딩용으로 따로 보관) ──
    train_seq_raw = full_train_df['seq'].copy() if 'seq' in full_train_df.columns else None
    test_seq_raw = test_df['seq'].copy() if 'seq' in test_df.columns else None
    if 'seq' in full_train_df.columns:
        full_train_df.drop(columns=['seq'], inplace=True)
    if 'seq' in test_df.columns:
        test_df.drop(columns=['seq'], inplace=True)
    print("✅ seq 파생 피처 생성 + seq 컬럼 제거 완료")

    # ── 4. Train / Validation Split ──
    print("📊 Splitting train/validation...")
    train_df, val_df = train_test_split(
        full_train_df, test_size=CFG.VALID_RATIO,
        random_state=seed, stratify=full_train_df['clicked']
    )
    # seq도 같이 split
    if train_seq_raw is not None:
        train_seq = train_seq_raw.loc[train_df.index]
        val_seq = train_seq_raw.loc[val_df.index]
    del full_train_df; gc.collect()

    # ── 5. Train set만 다운샘플링 ──
    print("🔻 Down-sampling train only...")
    clicked_1 = train_df[train_df['clicked'] == 1]
    n_neg = min(int(len(clicked_1) * CFG.NEG_RATIO),
                len(train_df[train_df['clicked'] == 0]))
    clicked_0 = train_df[train_df['clicked'] == 0].sample(n=n_neg, random_state=seed)
    
    # [FIX] 먼저 결합 및 셔플 (인덱스 유지)
    downsampled_df = pd.concat([clicked_1, clicked_0]).sample(frac=1, random_state=seed)
    
    # seq 데이터 동기화 (같은 인덱스 사용) 및 Series 유지
    if train_seq_raw is not None:
        train_seq = train_seq_raw.loc[downsampled_df.index].reset_index(drop=True)
        
    # train_df 최종 확정
    train_df = downsampled_df.reset_index(drop=True)
    
    print(f"  After downsampling: {train_df.shape}")

    # ── 6. Feature Engineering ──
    print("🛠️ Feature Engineering...")
    
    if CFG.USE_INTERACTION_FEATURES:
        print("  -> Adding interaction features...")
        train_df = add_interaction_features(train_df)
        val_df = add_interaction_features(val_df)
        test_df = add_interaction_features(test_df)
    else:
        print("  -> Skipping interaction features...")

    if CFG.USE_GROUPBY_FEATURES:
        print("  -> Adding groupby features...")
        train_df, val_df, test_df = add_groupby_features(
            train_df, val_df, test_df, 'inventory_id'
        )
    else:
        print("  -> Skipping groupby features...")

    if CFG.USE_COUNT_FEATURES:
        print("  -> Adding count features...")
        count_target_cols = ['gender', 'age_group', 'inventory_id', 'day_of_week', 'hour']
        count_target_cols = [c for c in count_target_cols if c in train_df.columns]
        train_df, val_df, test_df = add_count_features(train_df, val_df, test_df, count_target_cols)
    else:
        print("  -> Skipping count features...")

    # ── 7. 컬럼 분리: 수치형 / 범주형 ──
    exclude_cols = {'clicked', 'ID'}
    all_cols = [c for c in train_df.columns if c not in exclude_cols]

    # 범주형 = CFG.CATEGORICAL_COLS + object/string dtype (interaction 피처 포함)
    cat_cols = [
        c for c in all_cols
        if (c in CFG.CATEGORICAL_COLS or not pd.api.types.is_numeric_dtype(train_df[c]))
        and c in val_df.columns and c in test_df.columns
    ]
    # 수치형 = 나머지 (범주형 제외)
    num_cols = [c for c in all_cols if c not in cat_cols
                and c in val_df.columns and c in test_df.columns]

    print(f"  수치형 피처: {len(num_cols)}개, 범주형 피처: {len(cat_cols)}개")

    # ── 8. 전처리: 수치형 ──
    print("📋 Preprocessing numerical features...")
    # Scaling
    scaler = StandardScaler()
    train_df[num_cols] = scaler.fit_transform(train_df[num_cols])
    val_df[num_cols] = scaler.transform(val_df[num_cols])
    test_df[num_cols] = scaler.transform(test_df[num_cols])

    # NaN 처리 (train 기준 평균)
    num_imputer = SimpleImputer(strategy='mean')
    train_df[num_cols] = num_imputer.fit_transform(train_df[num_cols])
    val_df[num_cols] = num_imputer.transform(val_df[num_cols])
    test_df[num_cols] = num_imputer.transform(test_df[num_cols])

    # ── 9. 전처리: 범주형 (OneHotEncoder → CSR) ──
    print("📋 OneHot Encoding categorical features...")
    # 범주형 NaN → 문자열 처리
    cat_imputer = SimpleImputer(strategy='most_frequent')
    train_df[cat_cols] = cat_imputer.fit_transform(train_df[cat_cols])
    val_df[cat_cols] = cat_imputer.transform(val_df[cat_cols])
    test_df[cat_cols] = cat_imputer.transform(test_df[cat_cols])

    ohe = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
    train_cat_sparse = ohe.fit_transform(train_df[cat_cols])
    val_cat_sparse = ohe.transform(val_df[cat_cols])
    test_cat_sparse = ohe.transform(test_df[cat_cols])
    print(f"  OneHot shape: Train={train_cat_sparse.shape}, Val={val_cat_sparse.shape}, Test={test_cat_sparse.shape}")

    # ── 10. seq 임베딩 (선택 사항 — 사용하려면 아래 주석 해제) ──
    ### seq 임베딩 시작 ###
    # from src.models.seq_embedding import SeqEncoder
    # seq_encoder = SeqEncoder(embedding_dim=16)
    
    # # Train: 학습 + 벡터 추출
    # seq_train = seq_encoder.get_features(
    #     train_seq,
    #     target_series=train_df['clicked'],
    #     train_mode=True, epochs=5
    # )
    # # Valid/Test: 벡터만 추출
    # seq_val = seq_encoder.get_features(val_seq, train_mode=False)
    # seq_test = seq_encoder.get_features(test_seq_raw, train_mode=False)
    # ## seq 임베딩 끝 → 아래 None을 삭제하고 위 주석 해제 ###
    seq_train = None
    seq_val = None
    seq_test = None

    # ── 11. Feature 결합 (hstack) ──
    print("🔗 Combining features with hstack...")
    hstack_list_train = [train_df[num_cols], train_cat_sparse]
    hstack_list_val = [val_df[num_cols], val_cat_sparse]
    hstack_list_test = [test_df[num_cols], test_cat_sparse]

    if seq_train is not None:
        hstack_list_train.append(seq_train)
        hstack_list_val.append(seq_val)
        hstack_list_test.append(seq_test)

    X_train = pandas_hstack(hstack_list_train)
    y_train = train_df['clicked'].values
    X_val = pandas_hstack(hstack_list_val)
    y_val = val_df['clicked'].values
    X_test = pandas_hstack(hstack_list_test)

    print(f"  X_train: {X_train.shape}, X_val: {X_val.shape}, X_test: {X_test.shape}")

    

    # ── 12. 모델 학습 ──
    print("🔥 Training XGBoost...")
    model = xgb.XGBClassifier(**CFG.XGB_PARAMS, random_state=seed)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=50
    )

    # ── 13. 평가 ──
    print("📊 Evaluating...")
    eval_df = evaluate_model(model, X_train, y_train, X_val, y_val, threshold=0.5)
    print(eval_df.round(4).to_string())

    # ── 13-1. 평가 지표 CSV 저장 ──
    metrics_file = os.path.join(CFG.OUTPUT_DIR, f"{CFG.OUTPUT_PREFIX}_metrics_{seed}.csv")
    eval_df.to_csv(metrics_file)
    print(f"📊 Metrics saved: {metrics_file}")

    # ── 14. 예측 저장 ──
    test_pred = model.predict_proba(X_test)[:, 1]
    out_file = os.path.join(CFG.OUTPUT_DIR, f"{CFG.OUTPUT_PREFIX}_{seed}.csv")
    sub = pd.DataFrame({
        'ID': test_df['ID'].values if 'ID' in test_df.columns else range(len(test_pred)),
        'clicked': test_pred
    })
    sub.to_csv(out_file, index=False)
    prediction_files.append(out_file)
    print(f"💾 Saved: {out_file}")

    del train_df, val_df, model, X_train, X_val, X_test
    gc.collect()

print(f"\n✅ All seeds processed: {prediction_files}")


📄 Configuration saved: results/XGB_CPU_20260226_0729/XGB_CPU_20260226_0729_config.json

🚀 Processing Seed: 42
📂 Loading: /home/konkukstat/python3/workspace/data/train.parquet
✅ Train loaded: (10704179, 119)
✅ Test loaded: (1527298, 119)
🔄 Converting categorical columns to str...
✅ Categorical dtype conversion done
⏭️ Skipping seq features...
🛠️ Building item CTR map from train seq...
  ✅ Item CTR map built: 594 items
🛠️ Adding seq features v2 (nunique, click_score, repeat_ratio)...
✅ seq 파생 피처 생성 + seq 컬럼 제거 완료
📊 Splitting train/validation...
🔻 Down-sampling train only...
  After downsampling: (326686, 121)
🛠️ Feature Engineering...
  -> Adding interaction features...
  -> Adding groupby features...
  -> Adding count features...
  수치형 피처: 230개, 범주형 피처: 53개
📋 Preprocessing numerical features...
📋 OneHot Encoding categorical features...
  OneHot shape: Train=(326686, 1753), Val=(2140836, 1753), Test=(1527298, 1753)
🔗 Combining features with hstack...
  X_train: (326686, 1983), X_val: (21

[PART 6] Metrics Summary

In [8]:
print("\n📊 Creating metrics summary...")
all_metrics = []
for seed in CFG.SEEDS:
    metrics_file = os.path.join(CFG.OUTPUT_DIR, f"{CFG.OUTPUT_PREFIX}_metrics_{seed}.csv")
    if os.path.exists(metrics_file):
        df = pd.read_csv(metrics_file, index_col=0)
        df['Seed'] = seed
        all_metrics.append(df)

if all_metrics:
    summary_metrics = pd.concat(all_metrics, ignore_index=False)
    summary_file = os.path.join(CFG.OUTPUT_DIR, f"{CFG.OUTPUT_PREFIX}_metrics_summary.csv")
    summary_metrics.to_csv(summary_file)
    print(f"📊 Summary metrics saved: {summary_file}")
    print("\n" + summary_metrics.round(4).to_string())



📊 Creating metrics summary...
📊 Summary metrics saved: results/XGB_CPU_20260226_0729/XGB_CPU_20260226_0729_metrics_summary.csv

       ROC-AUC  PR-AUC  Avg Precision  Precision  Recall  F1-Score  Accuracy  Log Loss  Toss Score  Seed
Train   0.7854  0.7916         0.7916     0.7227  0.6878    0.7048    0.7120    0.5607      0.7162    42
Valid   0.7426  0.0800         0.0800     0.0409  0.6546    0.0770    0.7005    0.5922      0.3532    42
Train   0.7777  0.7836         0.7836     0.7149  0.6829    0.6985    0.7053    0.5670      0.7109   106
Valid   0.7415  0.0790         0.0790     0.0406  0.6569    0.0764    0.6970    0.5934      0.3525   106
Train   0.7841  0.7895         0.7895     0.7218  0.6881    0.7046    0.7115    0.5619      0.7149  1031
Valid   0.7403  0.0792         0.0792     0.0404  0.6517    0.0760    0.6978    0.5930      0.3525  1031


[PART 7] Soft Voting 앙상블

In [9]:
final_output = os.path.join(CFG.OUTPUT_DIR, f"{CFG.OUTPUT_PREFIX}_SoftVoting.csv")
soft_voting_ensemble(prediction_files, final_output)

print(f"\n{'='*60}")
print(f"🏁 모든 학습 및 앙상블 완료!")
print(f"  시드: {CFG.SEEDS}")
print(f"  개별 예측: {prediction_files}")
print(f"  앙상블 결과: {final_output}")
print(f"{'='*60}")


🎉 Ensemble saved: results/XGB_CPU_20260226_0729/XGB_CPU_20260226_0729_SoftVoting.csv
             ID   click_0   click_1   click_2   clicked
0  TEST_0000000  0.270375  0.353513  0.327055  0.316981
1  TEST_0000001  0.307034  0.276096  0.314368  0.299166
2  TEST_0000002  0.389529  0.451339  0.457040  0.432636
3  TEST_0000003  0.527451  0.540300  0.432835  0.500195
4  TEST_0000004  0.347424  0.426988  0.319743  0.364718

🏁 모든 학습 및 앙상블 완료!
  시드: [42, 106, 1031]
  개별 예측: ['results/XGB_CPU_20260226_0729/XGB_CPU_20260226_0729_42.csv', 'results/XGB_CPU_20260226_0729/XGB_CPU_20260226_0729_106.csv', 'results/XGB_CPU_20260226_0729/XGB_CPU_20260226_0729_1031.csv']
  앙상블 결과: results/XGB_CPU_20260226_0729/XGB_CPU_20260226_0729_SoftVoting.csv
